# Deploy SageMaker Serverless Endpoint

## Sentiment Binary Classification (fine-tuning with KoELECTRA-Small-v3 model and Naver Sentiment Movie Corpus dataset)

- KoELECTRA: https://github.com/monologg/KoELECTRA
- Naver Sentiment Movie Corpus Dataset: https://github.com/e9t/nsmc

---

## Overview

Amazon SageMaker Serverless Inference는 re:Invent 2021에 런칭된 신규 추론 옵션으로 호스팅 인프라 관리에 대한 부담 없이 머신 러닝을 모델을 쉽게 배포하고 확장할 수 있도록 제작된 신규 추론 옵션입니다. SageMaker Serverless Inference는 컴퓨팅 리소스를 자동으로 시작하고 트래픽에 따라 자동으로 스케일 인/아웃을 수행하므로 인스턴스 유형을 선택하거나 스케일링 정책을 관리할 필요가 없습니다. 따라서, 트래픽 급증 사이에 유휴 기간이 있고 콜드 스타트를 허용할 수 있는 워크로드에 이상적입니다.

## Difference from Lambda Serverless Inference


### Lambda Serverless Inference

- Lambda 컨테이너용 도커 이미지 빌드/디버그 후 Amazon ECR(Amazon Elastic Container Registry)에 푸시
- Option 1: Lambda 함수를 생성하여 직접 모델 배포 수행
- Option 2: SageMaker API로 SageMaker에서 모델 배포 수행 (`LambdaModel` 및 `LambdaPredictor` 리소스를 순차적으로 생성) 단, Option 2를 사용하는 경우 적절한 권한을 직접 설정해 줘야 합니다.
    - SageMaker과 연결된 role 대해 ECR 억세스를 허용하는 policy 생성 및 연결
    - SageMaker 노트북에서 lambda를 실행할 수 있는 role 생성
    - Lambda 함수가 ECR private 리포지토리에 연결하는 억세스를 허용하는 policy 생성 및 연결 


### SageMaker Serverless Inference

기존 Endpoint 배포 코드에서 Endpoint 설정만 변경해 주시면 되며, 별도의 도커 이미지 빌드가 필요 없기에 쉽고 빠르게 서버리스 추론을 수행할 수 있습니다.

**주의**
- 현재 서울 리전을 지원하지 않기 때문에 아래 리전 중 하나를 선택해서 수행하셔야 합니다.
    - 현재 지원하는 리전: US East (Northern Virginia), US East (Ohio), US West (Oregon), EU (Ireland), Asia Pacific (Tokyo) and Asia Pacific (Sydney)
- boto3, botocore, sagemaker, awscli는 2021년 12월 버전 이후를 사용하셔야 합니다.

<br>

## 1. Upload Model Artifacts
---

모델을 아카이빙하여 S3로 업로드합니다.

In [ ]:
!pip install -qU sagemaker botocore boto3 awscli
!pip install --ignore-installed PyYAML
!pip install transformers==4.12.5

In [ ]:
import torch
import torchvision
import torchvision.models as models
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
import boto3
import datetime
import time
from time import strftime,gmtime
import json
import os
import io
import torchvision.transforms as transforms
from src.utils import print_outputs, upload_model_artifact_to_s3, NLPPredictor 

role = get_execution_role()
boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
region = boto_session.region_name
bucket = sm_session.default_bucket()
prefix = 'serverless-inference-kornlp-nsmc'

print(f'region = {region}')
print(f'role = {role}')
print(f'bucket = {bucket}')
print(f'prefix = {prefix}')

In [ ]:
model_variant = 'modelA'
nlp_task = 'nsmc'
model_path = f'model-{nlp_task}'
model_s3_uri = upload_model_artifact_to_s3(model_variant, model_path, bucket, prefix)

<br>

## 2. Create SageMaker Serverless Endpoint
---

SageMaker Serverless Endpoint는 기존 SageMaker 리얼타임 엔드포인트 배포와 99% 유사합니다. 1%의 차이가 무엇일까요? Endpoint 구성 설정 시, ServerlessConfig에서 메모리 크기(`MemorySizeInMB`), 최대 동시 접속(`MaxConcurrency`)에 대한 파라메터만 추가하시면 됩니다.

```python
sm_client.create_endpoint_config(
  ...
  "ServerlessConfig": {
    "MemorySizeInMB": 2048,
    "MaxConcurrency": 20
  }
)
```

자세한 내용은 아래 링크를 참조해 주세요.
- Amazon SageMaker Developer Guide - Serverless Inference: https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html

### Create Inference containter definition for Model

In [ ]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.m5.xlarge'
pt_ecr_image_uri = retrieve(
    framework='pytorch',
    region=region,
    version='1.7.1',
    py_version='py3',
    instance_type = deploy_instance_type,
    accelerator_type=None,
    image_scope='inference'
)

### Create a SageMaker Model

`create_model` API를 호출하여 위 코드 셀에서 생성한 컨테이너의 정의를 포함하는 모델을 생성합니다.

In [ ]:
model_name = f"KorNLPServerless-{nlp_task}-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

create_model_response = sm_client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": pt_ecr_image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_s3_uri,
            "Environment": {
                "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                "SAGEMAKER_PROGRAM": "inference_nsmc.py",
                "SAGEMAKER_SUBMIT_DIRECTORY": model_s3_uri,
            },                
        }        
        
    ],
    ExecutionRoleArn=role,
)
print(f"Created Model: {create_model_response['ModelArn']}")

### Create Endpoint Configuration

`ServerlessConfig`으로 엔드포인트에 대한 서버리스 설정을 조정할 수 있습니다. 최대 동시 호출(`MaxConcurrency`; max concurrent invocations)은 1에서 50 사이이며, `MemorySize`는 1024MB, 2048MB, 3072MB, 4096MB, 5120MB 또는 6144MB를 선택할 수 있습니다.

In [ ]:
endpoint_config_name = f"KorNLPServerlessEndpointConfig-{nlp_task}-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 20,
            },            
        },
    ],
)
print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

### Create a SageMaker Multi-container endpoint

create_endpoint API로 멀티 컨테이너 엔드포인트를 생성합니다. 기존의 엔드포인트 생성 방법과 동일합니다.

In [ ]:
endpoint_name = f"KorNLPServerlessEndpoint-{nlp_task}-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)
print(f"Creating Endpoint: {endpoint_response['EndpointArn']}")

`describe_endpoint` API를 사용하여 엔드포인트 생성 상태를 확인할 수 있습니다. SageMaker 서버리스 엔드포인트는 일반적인 엔드포인트 생성보다 빠르게 생성됩니다. (약 2-3분)

In [ ]:
%%time
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

### Direct Invocation for Model 

최초 호출 시 Cold start로 지연 시간이 발생하지만, 최초 호출 이후에는 warm 상태를 유지하기 때문에 빠르게 응답합니다. 물론 수 분 동안 호출이 되지 않거나 요청이 많아지면 cold 상태로 바뀐다는 점을 유의해 주세요.

In [ ]:
model_sample_path = 'samples/nsmc.txt'
!cat $model_sample_path
with open(model_sample_path, mode='rb') as file:
    model_input_data = file.read()  

model_response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    Body=model_input_data
)

model_outputs = model_response['Body'].read().decode()
print()
print_outputs(model_outputs)

### Check Model Latency

In [ ]:
import time
start = time.time()
for _ in range(10):
    model_response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    Body=model_input_data
)
inference_time = (time.time()-start)
print(f'Inference time is {inference_time:.4f} ms.')

## Clean Up

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)